In [1]:
# Import dependencies
import pandas as pd
import pprint
from inspect import getmembers

In [2]:
# Test functionality of Boxscore module
from sportsipy.nfl.boxscore import Boxscores

games_today = Boxscores(1, 2017)
# Prints a dictionary of all matchups for week 1 of 2017
pprint.pprint(games_today.games)

{'1-2017': [{'away_abbr': 'kan',
             'away_name': 'Kansas City Chiefs',
             'away_score': 42,
             'boxscore': '201709070nwe',
             'home_abbr': 'nwe',
             'home_name': 'New England Patriots',
             'home_score': 27,
             'losing_abbr': 'nwe',
             'losing_name': 'New England Patriots',
             'winning_abbr': 'kan',
             'winning_name': 'Kansas City Chiefs'},
            {'away_abbr': 'atl',
             'away_name': 'Atlanta Falcons',
             'away_score': 23,
             'boxscore': '201709100chi',
             'home_abbr': 'chi',
             'home_name': 'Chicago Bears',
             'home_score': 17,
             'losing_abbr': 'chi',
             'losing_name': 'Chicago Bears',
             'winning_abbr': 'atl',
             'winning_name': 'Atlanta Falcons'},
            {'away_abbr': 'nyj',
             'away_name': 'New York Jets',
             'away_score': 12,
             'boxscore': '201

In [3]:
from sportsipy.nfl.boxscore import Boxscore

game_data = Boxscore('201802040nwe')
#print(game_data.home_points)  # Prints 33
#print(game_data.away_points)  # Prints 41
df = game_data.dataframe  # Returns a Pandas DataFrame of game metrics
df

,attendance,away_first_downs,away_fourth_down_attempts,away_fourth_down_conversions,away_fumbles,away_fumbles_lost,away_interceptions,away_net_pass_yards,away_pass_attempts,away_pass_completions,...,roof,stadium,surface,time,vegas_line,weather,winner,winning_abbr,winning_name,won_toss
201802040nwe,67612,25,2,2,0,0,1,374,44,29,...,Dome,U.S. Bank Stadium,Sportturf,6:30pm,New England Patriots -4.5,None,Away,PHI,Philadelphia Eagles,Patriots (deferred)


In [4]:
# Test functionality of Player module
from sportsipy.nfl.roster import Player

pprint.pprint(getmembers(Player))

[('__call__', <function Player.__call__ at 0x000002294A8455E8>),
 ('__class__', <class 'type'>),
 ('__delattr__', <slot wrapper '__delattr__' of 'object' objects>),
 ('__dict__',
  mappingproxy({'__call__': <function Player.__call__ at 0x000002294A8455E8>,
                '__doc__': '\n'
                           '    Get player information and stats for all '
                           'seasons.\n'
                           '\n'
                           "    Given a player ID, such as 'BreeDr00' for Drew "
                           'Brees, capture all relevant\n'
                           '    stats and information like name, team, '
                           'height/weight, career starts, single\n'
                           '    season pasing yards, sacks, and much more.\n'
                           '\n'
                           '    By default, the class instance will return the '
                           "player's career stats, but\n"
                           '    si

In [5]:
pprint.pprint(dir(Player.position))

['__class__',
 '__delattr__',
 '__delete__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__isabstractmethod__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__set__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'deleter',
 'fdel',
 'fget',
 'fset',
 'getter',
 'setter']


In [6]:
from sportsipy.nfl.teams import Teams
teams2021 = Teams(year = '2021')
print(dir(teams2021))

['__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_instantiate_teams', '_teams', 'dataframes']


In [7]:
for team in teams2021:
    print(team.name, 'Points Scored:', team.points_for)

Dallas Cowboys Points Scored: 205
Buffalo Bills Points Scored: 203
Tampa Bay Buccaneers Points Scored: 195
Arizona Cardinals Points Scored: 194
Kansas City Chiefs Points Scored: 185
Los Angeles Rams Points Scored: 179
Baltimore Ravens Points Scored: 170
Tennessee Titans Points Scored: 166
Cleveland Browns Points Scored: 156
Los Angeles Chargers Points Scored: 148
Cincinnati Bengals Points Scored: 148
Minnesota Vikings Points Scored: 147
Las Vegas Raiders Points Scored: 147
Green Bay Packers Points Scored: 144
Carolina Panthers Points Scored: 143
Seattle Seahawks Points Scored: 140
Indianapolis Colts Points Scored: 139
Philadelphia Eagles Points Scored: 137
Washington Football Team Points Scored: 136
New Orleans Saints Points Scored: 127
Denver Broncos Points Scored: 126
New England Patriots Points Scored: 125
Pittsburgh Steelers Points Scored: 117
San Francisco 49ers Points Scored: 117
Jacksonville Jaguars Points Scored: 116
New York Giants Points Scored: 114
Detroit Lions Points Score

In [8]:
from sportsipy.nfl.schedule import Schedule
dal2021 = Schedule('DAL', year = '2021')
dal2021.dataframe

,boxscore_index,date,datetime,day,extra_points_attempted,extra_points_made,field_goals_attempted,field_goals_made,fourth_down_attempts,fourth_down_conversions,...,rush_touchdowns,rush_yards,rush_yards_per_attempt,third_down_attempts,third_down_conversions,time_of_possession,times_sacked,type,week,yards_lost_from_sacks
202109090tam,202109090tam,September 9,2021-09-09,Thu,3,2,5,3,0,0,...,0,60,3.3,17,9,34:27,1,Reg,1,12
202109190sdg,202109190sdg,September 19,2021-09-19,Sun,2,2,2,2,1,0,...,2,198,6.4,10,6,29:03,2,Reg,2,16
202109270dal,202109270dal,September 27,2021-09-27,Mon,6,5,0,0,2,1,...,2,160,3.9,12,6,34:51,4,Reg,3,18
202110030dal,202110030dal,October 3,2021-10-03,Sun,3,3,1,1,1,1,...,1,245,7.2,9,3,26:24,0,Reg,4,0
202110100dal,202110100dal,October 10,2021-10-10,Sun,5,5,3,3,2,1,...,1,201,5.2,14,8,32:12,2,Reg,5,10
202110170nwe,202110170nwe,October 17,2021-10-17,Sun,2,2,4,3,4,2,...,0,122,3.9,13,3,39:17,0,Reg,6,0
202110310min,202110310min,October 31,2021-10-31,Sun,None,None,None,None,None,None,...,None,None,None,None,None,,None,Reg,8,None
202111070dal,202111070dal,November 7,2021-11-07,Sun,None,None,None,None,None,None,...,None,None,None,None,None,,None,Reg,9,None
202111140dal,202111140dal,November 14,2021-11-14,Sun,None,None,None,None,None,None,...,None,None,None,None,None,,None,Reg,10,None
202111210kan,202111210kan,November 21,2021-11-21,Sun,None,None,None,None,None,None,...,None,None,None,None,None,,None,Reg,11,None


In [9]:
game_data_dal = Boxscore('202109090tam')
print(dir(game_data_dal))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_alt_abbreviations', '_attendance', '_away_abbr', '_away_first_downs', '_away_fourth_down_attempts', '_away_fourth_down_conversions', '_away_fumbles', '_away_fumbles_lost', '_away_interceptions', '_away_name', '_away_net_pass_yards', '_away_pass_attempts', '_away_pass_completions', '_away_pass_touchdowns', '_away_pass_yards', '_away_penalties', '_away_players', '_away_points', '_away_rush_attempts', '_away_rush_touchdowns', '_away_rush_yards', '_away_third_down_attempts', '_away_third_down_conversions', '_away_time_of_possession', '_away_times_sacked', '_away_total_yards', '_away_turnovers', '_away_yards_from_penalties', '_away_yards_lost_from_sa

In [10]:
game_data_dal.dataframe

,attendance,away_first_downs,away_fourth_down_attempts,away_fourth_down_conversions,away_fumbles,away_fumbles_lost,away_interceptions,away_net_pass_yards,away_pass_attempts,away_pass_completions,...,roof,stadium,surface,time,vegas_line,weather,winner,winning_abbr,winning_name,won_toss
202109090tam,65566,30,0,0,1,0,1,391,58,42,...,Outdoors,Raymond James Stadium,Grass,8:20pm,Tampa Bay Buccaneers -8.5,"82 degrees, relative humidity 85%, wind 9 mph",Home,TAM,Tampa Bay Buccaneers,Cowboys (deferred)


In [11]:
game_data_dal.home_players

In [12]:
print(dir(game_data.home_players[0]))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_assists_on_tackles', '_attempted_passes', '_average_kickoff_return_yards', '_combined_tackles', '_completed_passes', '_extra_points_attempted', '_extra_points_made', '_field_goals_attempted', '_field_goals_made', '_fumbles', '_fumbles_forced', '_fumbles_lost', '_fumbles_recovered', '_fumbles_recovered_for_touchdown', '_index', '_interceptions', '_interceptions_returned_for_touchdown', '_interceptions_thrown', '_kickoff_return_touchdown', '_kickoff_return_yards', '_kickoff_returns', '_longest_interception_return', '_longest_kickoff_return', '_longest_pass', '_longest_punt', '_longest_punt_return', '_longest_reception', '_longest_rush', '_name', '

In [13]:
home_df_dal = game_data_dal.home_players[0].dataframe
for player in game_data_dal.home_players[1:]:
    home_df_dal = pd.concat([home_df_dal, player.dataframe], axis = 0)
home_df_dal['name'] = [x.name for x in game_data_dal.home_players]
home_df_dal.set_index('name', inplace = True)
home_df_dal.dropna()
home_df_dal

,completed_passes,attempted_passes,passing_yards,passing_touchdowns,interceptions_thrown,times_sacked,yards_lost_from_sacks,longest_pass,quarterback_rating,rush_attempts,...,punt_return_touchdown,longest_punt_return,extra_points_made,extra_points_attempted,field_goals_made,field_goals_attempted,punts,total_punt_yards,yards_per_punt,longest_punt
name,,,,,,,,,,,,,,,,,,,,,
Tom Brady,32,50,379,4,2,0,0,47,97.0,0,...,None,None,None,None,None,None,None,None,None,None
Leonard Fournette,0,0,0,0,0,0,0,0,NaN,9,...,None,None,None,None,None,None,None,None,None,None
Ronald Jones II,0,0,0,0,0,0,0,0,NaN,4,...,None,None,None,None,None,None,None,None,None,None
Antonio Brown,0,0,0,0,0,0,0,0,NaN,1,...,None,None,None,None,None,None,None,None,None,None
Chris Godwin,0,0,0,0,0,0,0,0,NaN,0,...,None,None,None,None,None,None,None,None,None,None
Rob Gronkowski,0,0,0,0,0,0,0,0,NaN,0,...,None,None,None,None,None,None,None,None,None,None
Mike Evans,0,0,0,0,0,0,0,0,NaN,0,...,None,None,None,None,None,None,None,None,None,None
Giovani Bernard,0,0,0,0,0,0,0,0,NaN,0,...,None,None,None,None,None,None,None,None,None,None
Tyler Johnson,0,0,0,0,0,0,0,0,NaN,0,...,None,None,None,None,None,None,None,None,None,None


In [47]:
from sportsipy.nfl.roster import Roster

dallas = Roster('DAL')

for player in dallas.players:
    print(player.name)

In [14]:
from sportsipy.nfl.roster import Roster

saints = Roster('NOR')
for player in saints.players:
    # Prints the name of all players who played for the New Orleans Saints
    # in the most recent season.
    print(player.name)

In [16]:
from sportsipy.nfl.schedule import Schedule

dallas_schedule = Schedule('DAL')
for game in dallas_schedule:
    print(game.boxscore_index)  # Prints the boxscore URI for each game
    boxscore = game.boxscore  # Returns an instance of the Boxscore class for this specific game


202109090tam
202109190sdg
202109270dal
202110030dal
202110100dal
202110170nwe
202110310min
202111070dal
202111140dal
202111210kan
202111250dal
202112020nor
202112120was
202112190nyg
202112260dal
202201020dal
202201090phi


In [18]:
from sportsipy.nhl.roster import Roster

detroit = Roster('DET')
for player in detroit.players:
    print(player.name, player.games_played)

Tyler Bertuzzi 211
Danny DeKeyser 491
Adam Erne 114
Robby Fabbri 164
Sam Gagner 542
Thomas Greiss None
Filip Hronek 170
Dylan Larkin 435
Nick Leddy 518
Gustav Lindstrom 32
Vladislav Namestnikov 263
Alex Nedeljkovic None
Jordan Oesterle 25
Michael Rasmussen 105
Lucas Raymond 3
Carter Rowney 152
Moritz Seider 3
Givani Smith 40
Marc Staal 892
Troy Stecher 286
Mitchell Stephens 45
Pius Suter 55
Filip Zadina 89
